In [486]:
import json, os
import pandas as pd
from nltk.corpus import stopwords
import numpy as np
from pymorphy2 import MorphAnalyzer
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer
morph = MorphAnalyzer()
stops = set(stopwords.words('russian'))

In [4]:
pd.set_option('display.max_colwidth', 1000)

## Данные

Возьмем данные вот отсюда - https://github.com/mannefedov/ru_kw_eval_datasets Там лежат 4 датасета (статьи с хабра, с Russia Today, Независимой газеты и научные статьи с Киберленинки). Датасет НГ самый маленький, поэтому возьмем его в качестве примера.

In [5]:
# скачаем данные в папке data и распакуем их
PATH_TO_DATA = '/Users/irene/Downloads/autobreya/data'

In [6]:
files = [os.path.join(PATH_TO_DATA, file) for file in os.listdir(PATH_TO_DATA)]

In [7]:
files

['/Users/irene/Downloads/autobreya/data/ng_0.jsonlines',
 '/Users/irene/Downloads/autobreya/data/ng_1.jsonlines']

Объединим файлы в один датасет.

In [8]:
data = pd.concat([pd.read_json(file, lines=True) for file in files], axis=0, ignore_index=True)

In [9]:
data.shape

(1987, 5)

In [10]:
data.head(3)

,content,keywords,summary,title,url
0,"Многие интересуются, зачем нужна «Яблоку» молодежная фракция? Основной задачей «Молодежного «Яблока» является привлечение молодых людей к участию в выборах и деятельности партии. «Молодежное «Яблоко» работает более чем в 10 регионах. Единого руководства у нас нет, но мы стараемся координировать свою деятельность и периодически проводим акции на федеральном уровне.\nМы ведем борьбу с обязательным воинским призывом. Военный – это профессия, а не обязанность. Молодые люди вправе сами распоряжаться своей жизнью и не терять целый год, отдавая государству «долг», который они у него не занимали. По мнению одного из ведущих специалистов в области оборонной политики Алексея Арбатова, переход на контрактную армию будет стоить лишь 2% военного бюджета.\nТакже на федеральном уровне «Молодежное «Яблоко» проводило акции за освобождение политзаключенных и против вмешательства России во внутреннюю политику Украины.\nРасскажу о московских активистах. Виктору Петрунину – 19 лет, он пришел к нам боль...","[яблоко, молодежь, молодежное яблоко]",,"""Молодежное ""Яблоко"": оппозиционная деятельность становится опасной",http://www.ng.ru/ng_politics/2017-04-18/11_6976_apple.html
1,"Вчера «Газпром» снизил верхнюю планку прогноза собственной добычи газа в 2020 году. Через 12 лет концерн собирается добывать около 620–640 млрд. куб. м в год. При этом общее производство газа в стране, по расчетам холдинга, должно достичь 940 млрд. куб. м. Иными словами, треть добываемого объема, по мнению холдинга, должны будут обеспечить независимые производители. Эксперты не верят, что независимые компании смогут выйти на такие объемы добычи. Если расчеты «Газпрома» не оправдаются, то под ударом окажутся отечественные предприятия и население, которым придется сокращать потребление и смириться с новым витком цен. Иных путей покрытия возможного дефицита газа нет, так как вряд ли холдинг разорвет уже заключенные контракты на экспорт газа в другие страны. \n«Газпром» к 2020 году планирует добывать 620–640 млрд. куб. м газа, сообщил вчера на форуме «ТЭК России в ХХI веке» глава управления по добыче газа, газового конденсата и нефти холдинга Валерий Минликаев. Тем самым он уточнил пре...","[газпром, газ]",,"""Газпрома"" на всех не хватит",http://www.ng.ru/economics/2008-04-03/1_gazprom.html
2,"Долголетний труд Евгения Витковского на ниве перевода, а также в качестве редактора и антологиста известен многим. Но не все знают его как поэта и прозаика. В этом году уже вышла составленная им и Еленой Кистеровой антология «Раздол туманов: Страницы шотландской гэльской поэзии XVII–XX вв.», а в апреле запланирован выход его романа «Протей, или Византийский кризис» (отрывок из романа читайте на с. 12). С \n побеседовал \n– Одна из таких книг только что вышла – «Раздол туманов. Страницы шотландской гэльской поэзии XVII–XX веков». Это стихи 29 поэтов, все в переводе с оригинала – моем и Елены Кистеровой. Работа заняла 10 лет, включая изучение языка. Она была упоительно интересной: до нас переводов из этой поэзии на русский не было вовсе. Сейчас должен выйти том стихотворений канадского классика Роберта Уильяма Сервиса, «канадского Киплинга», около 300 стихотворений. Кроме того, в Петербурге в производстве наш огромный трехтомный плод совместной работы – антология «Франция в сердце»....","[франсуа рабле, сервантес, шекспир, конан дойл, михаил булгаков, александр грин, борхес, босх, маркес, герман гессе, голландская живопись, гаргантюа и пантагрюэль, дон кихот, мастер и маргарита, москва, россия, история, поэзия, шотландия, баллада, пере]","Евгений Витковский о том, как Босх протягивает руку Шекспиру, \r\nи оба танцуют в пламени пожара в охваченном чумой средневековом городе",Бесконечная партия в четырехмерные шахматы,http://www.ng.ru/person/2018-03-22/10_927_vitkovsky.html


Каждой статье приписано какое-то количество ключевых слов. Допустим, что это единственно правильный набор ключевых слов (что конечно не так, но других данных у нас нет). Наша задача - придумать как извлекать точно такой же список автоматически.  
Зададим несколько метрик, по которым будем определять качество извлекаемых ключевых слов - точность, полноту, ф1-меру и меру жаккарда.

In [11]:
def evaluate(true_kws, predicted_kws):
    assert len(true_kws) == len(predicted_kws)
    
    precisions = []
    recalls = []
    f1s = []
    jaccards = []
    
    for i in range(len(true_kws)):
        true_kw = set(true_kws[i])
        predicted_kw = set(predicted_kws[i])
        
        tp = len(true_kw & predicted_kw)
        union = len(true_kw | predicted_kw)
        fp = len(predicted_kw - true_kw)
        fn = len(true_kw - predicted_kw)
        
        if (tp+fp) == 0:
            prec = 0
        else:
            prec = tp / (tp + fp)
        
        if (tp+fn) == 0:
            rec = 0
        else:
            rec = tp / (tp + fn)
        if (prec+rec) == 0:
            f1 = 0
        else:
            f1 = (2*(prec*rec))/(prec+rec)
            
        jac = tp / union
        
        precisions.append(prec)
        recalls.append(rec)
        f1s.append(f1)
        jaccards.append(jac)
    print('Precision - ', round(np.mean(precisions), 2))
    print('Recall - ', round(np.mean(recalls), 2))
    print('F1 - ', round(np.mean(f1s), 2))
    print('Jaccard - ', round(np.mean(jaccards), 2))
    
    
        

Проверим, что всё работает как надо.

In [12]:
evaluate(data['keywords'], data['keywords'])

Precision -  1.0
Recall -  1.0
F1 -  1.0
Jaccard -  1.0


# Тупое решение.

Давайте не будем думать, а попробуем сразу придумать какое-то решение.

Возьмем первые 5 слов из заголовка.

In [13]:
evaluate(data['keywords'], data['title'].apply(lambda x: x.lower().split()[:5]))

Precision -  0.06
Recall -  0.05
F1 -  0.05
Jaccard -  0.03


Или 10.

In [14]:
evaluate(data['keywords'], data['title'].apply(lambda x: x.lower().split()[:10]))

Precision -  0.06
Recall -  0.06
F1 -  0.05
Jaccard -  0.03


Теперь попробуем взять самые частотные слова.

In [15]:
evaluate(data['keywords'], data['content'].apply(lambda x: 
                                                 [x[0] for x in Counter(x.lower().split()).most_common(10)]))

Precision -  0.02
Recall -  0.04
F1 -  0.02
Jaccard -  0.01


Или вообще рандомные слова.

In [16]:
evaluate(data['keywords'], data['content'].apply(lambda x: 
                                                 np.random.choice(list(set(x.lower().split())), 10)))

Precision -  0.0
Recall -  0.01
F1 -  0.0
Jaccard -  0.0


Теперь давайте посмотрим, что вообще извлекается.

In [17]:
data['title'].apply(lambda x: x.lower().split()[:10]).head(10)

0                         ["молодежное, "яблоко":, оппозиционная, деятельность, становится, опасной]
1                                                                 ["газпрома", на, всех, не, хватит]
2                                                   [бесконечная, партия, в, четырехмерные, шахматы]
3    [экс-депутат,, осужденная, за, фальсификацию, выборов,, оказалась, членом, "боевого, братства"]
4                               [новая, москва, останется, территорией, экологической, безопасности]
5                                [f1., гран-при, сша, прошел, без, четырех, машин, и, со, «стопкой»]
6                                          [100, ведущих, политиков, россии, в, феврале, 2018, года]
7                                               [закон, "о, культуре", принимают, на, фоне, арестов]
8                                    [насколько, реальна, газовая, подоплека, сирийского, конфликта]
9                                  [фсб:, в, калужской, области, задержаны, четверо, участн

In [18]:
data['content'].apply(lambda x: [x[0] for x in Counter(x.lower().split()).most_common(10)]).head(10)

0                                                      [в, и, на, не, что, –, его, «молодежное, с, это]
1                                                            [в, и, на, –, млрд., куб., по, к, газа, м]
2                                                                 [в, –, и, не, я, но, что, это, на, с]
3                                                       [в, на, и, ким, по, –, что, видео, он, зинаиды]
4                                              [в, и, на, новой, площадью, москвы, –, развития, с, для]
5                                                             [в, на, и, не, с, но, уже, что, у, гонки]
6                                                  [на, в, (с, место)., и, рф, позиции, влияние, по, с]
7                                                        [в, и, –, по, с, культуре, не, из, будет, как]
8                                                              [в, и, на, с, что, для, по, –, не, газа]
9    [в, рф, террористической, организации, задержаны, –, четвер

Когда извлекаются частотные слова, то список почти полностью состоит из всяких стоп-слов. Также из-за плохой токенизации некоторые слова в обоих списках - пунктуация или слова с пунктуацией на концах. К тому же извлекаемые слова ненормализованы, а правильные ключевые слова - наоборот.

## Токенизация, удаление стоп-слов и нормализация.

In [19]:
from string import punctuation
from nltk.corpus import stopwords
punct = punctuation+'«»—…“”*№–'
stops = set(stopwords.words('russian'))

def normalize(text):
    
    words = [word.strip(punct) for word in text.lower().split()]
    words = [morph.parse(word)[0].normal_form for word in words if word and word not in stops]

    return words

In [20]:
data['content_norm'] = data['content'].apply(normalize)

In [21]:
data['title_norm'] = data['title'].apply(normalize)

In [22]:
data['title_norm'].head(10)

0            [молодёжный, яблоко, оппозиционный, деятельность, становиться, опасный]
1                                                                 [газпром, хватить]
2                                      [бесконечный, партия, четырехмерный, шахматы]
3    [экс-депутат, осудить, фальсификация, выбор, оказаться, член, боевой, братство]
4                 [новый, москва, остаться, территория, экологический, безопасность]
5                         [f1, гран-при, сша, пройти, четыре, машина, стопка, штраф]
6                                [100, ведущий, политик, россия, февраль, 2018, год]
7                                           [закон, культура, принимать, фон, арест]
8                     [насколько, реальный, газовый, подоплёка, сирийский, конфликт]
9                       [фсб, калужский, область, задержать, четверо, участник, иго]
Name: title_norm, dtype: object

Попробуем те же самые методы.

In [23]:
# топ 10 частотных слов статьи
evaluate(data['keywords'], data['content_norm'].apply(lambda x: [x[0] for x in Counter(x).most_common(10)]))

Precision -  0.11
Recall -  0.22
F1 -  0.14
Jaccard -  0.08


In [24]:
evaluate(data['keywords'],data['title_norm'].apply(lambda x: x[:10]))

Precision -  0.13
Recall -  0.13
F1 -  0.12
Jaccard -  0.07


Качество сильно улучшилось! Можно теперь ещё раз посмотреть, что плохого извлекается.

In [25]:
data['content_norm'].apply(lambda x: [x[0] for x in Counter(x).most_common(10)]).head(20)

0                         [яблоко, молодёжный, который, акция, год, активист, это, деятельность, политика, наш]
1                               [миллиард, газа, год, куб, метр, газпром, добыча, 2020, должный, производитель]
2                                              [год, это, книга, роман, тот, писать, выйти, один, мир, перевод]
3                                    [ким, зинаида, видео, год, журналист, суд, дело, бывший, футиный, который]
4                         [площадь, территория, новый, москва, га, который, столица, тинао, парковый, развитие]
5                                  [гонка, который, команда, место, позиция, два, один, из-за, круг, чемпионат]
6                        [место, влияние, рф, позиция, глава, россия, президент, сергей, политический, рейтинг]
7                 [культура, закон, который, сфера, стд, разработать, концепция, проект, изменение, сообщество]
8                              [газопровод, сирия, год, турция, газа, россия, европа, катар, который, ту

Ещё остались некоторые стоп-слова. Вместо того, чтобы расширять список, давайте попробуем выкинуть несуществительные.

In [26]:
def normalize(text):
    
    words = [word.strip(punct) for word in text.lower().split()]
    words = [morph.parse(word)[0] for word in words if word and word not in stops]
    words = [word.normal_form for word in words if word.tag.POS == 'NOUN']

    return words

In [27]:
data['content_norm'] = data['content'].apply(normalize)

In [28]:
evaluate(data['keywords'], data['content_norm'].apply(lambda x: [x[0] for x in Counter(x).most_common(10)]))

Precision -  0.13
Recall -  0.25
F1 -  0.16
Jaccard -  0.1


Ещу улучшения!

In [29]:
data['content_norm'].apply(lambda x: [x[0] for x in Counter(x).most_common(10)]).head(10)

0        [яблоко, акция, год, активист, деятельность, политика, власть, задача, молодая, человек]
1               [миллиард, газа, год, куб, метр, газпром, добыча, производитель, страна, прогноз]
2                [год, книга, роман, мир, перевод, стихотворение, читатель, жанр, поэзия, работа]
3                     [ким, зинаида, видео, год, журналист, суд, дело, рубль, процесс, заседание]
4                   [площадь, территория, москва, га, столица, тинао, развитие, парка, парк, год]
5                    [гонка, команда, место, позиция, круг, чемпионат, пилот, бокс, заезд, льюис]
6               [место, влияние, рф, позиция, глава, россия, президент, сергей, рейтинг, участие]
7    [культура, закон, сфера, концепция, проект, изменение, сообщество, услуга, учреждение, дело]
8                    [газопровод, сирия, год, турция, газа, россия, европа, катар, поток, проект]
9                 [участник, рф, организация, государство, область, центр, связь, фсб, март, год]
Name: content_norm, 

Не очень значимые слова все ещё остались. Давайте попробуем отсеять стоп-слова с помощью tfidf.

Воспользуемся TfidfVectorizer.

In [30]:
data['content_norm_str'] = data['content_norm'].apply(' '.join)

In [31]:
# можно заодно сделать нграммы
tfidf = TfidfVectorizer(ngram_range=(1,2), min_df=5)

In [32]:
tfidf.fit(data['content_norm_str'])

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=5,
        ngram_range=(1, 2), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words=None, strip_accents=None, sublinear_tf=False,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)

In [33]:
id2word = {i:word for i,word in enumerate(tfidf.get_feature_names())}

Преобразуем наши тексты в векторы, где на позиции i стоит tfidf коэффициент слова i из словаря.

In [34]:
texts_vectors = tfidf.transform(data['content_norm_str'])

Отсортируем векторы текстов по этим коэффициентам и возьмем топ-10.

In [35]:
# сортировка по убыванию, поэтому нужно развернуть список
keywords = [[id2word[w] for w in top] for top in texts_vectors.toarray().argsort()[:,:-11:-1]] 

In [36]:
keywords[:3]

[['яблоко',
  'активист',
  'акция',
  'дарья',
  'деятельность',
  'молодая человек',
  'политика',
  'виктор',
  'тимур',
  'репрессия'],
 ['миллиард куб',
  'куб метр',
  'куб',
  'газпром',
  'газа',
  'миллиард',
  'добыча',
  'добыча газа',
  'метр',
  'холдинг'],
 ['роман',
  'книга',
  'жанр',
  'стихотворение',
  'читатель',
  'перевод',
  'год',
  'поэзия',
  'произведение',
  'том']]

In [37]:
evaluate(data['keywords'], keywords)

Precision -  0.13
Recall -  0.25
F1 -  0.16
Jaccard -  0.09


Результат ещё немного улучшился. Немного подросла точность. Теперь вместо стоп-слов в ключевые попадают имена и все такое. Иногда это хорощо, а иногда нет (собянин - может быть ключевым словом, а дарья - вряд ли)

Возьмем этот результат за baseline. 

Precision -  0.13
Recall -  0.24
F1 -  0.16
Jaccard -  0.09

## Попробуем графы!

Большая часть методов для извлечения ключевых слов основана на применении графов. Основная идея - каким-то образом перевести текст в граф, а затем каким-то образом расчитать важность каждого узла и вывести топ-N самых важных узлов.  

Перевод текста в граф -  не тривиальная задача. Часто применяют такой подход - построим матрицу совстречаемости слов (в каком-то окне), эта матрица будет нашей матрицей смежности.

Для выбора важных узлов часто используют простой randow walk. Алгоритм примерно такой:  
1) Каким-то образом выбирается первый узел графа (например, случайно из равномерного распределения)  
2) на основе связей этого узла с другими, выбирается следующий узел  
3) шаг два повторяется некоторое количество раз (например, тысячу) __*чтобы не зацикливаться, с какой-то вероятностью мы случайно перескакиваем на другой узел (даже если он никак не связан с текущим, как в шаге 1)__  
5) на каждом шаге мы сохраняем узел в котором находимся  
6) в конце мы считаем в каких узлах мы были чаще всего и выводим top-N  


Предполагается, что мы часто будем приходить в важные узлы графа.

In [38]:
from itertools import combinations

Для наглядности реализуем этот подход без networkx. 

In [677]:
def get_kws(text, top=5, window_size=5, random_p=0.1, walks=500):

    vocab = set(text)
    word2id = {w:i for i, w in enumerate(vocab)}
    id2word = {i:w for i, w in enumerate(vocab)}
    # преобразуем слова в индексы для удобства
    ids = [word2id[word] for word in text]

    # создадим матрицу совстречаемости
    m = np.zeros((len(vocab), len(vocab)))

    # пройдемся окном по всему тексту
    for i in range(0, len(ids), window_size):
        window = ids[i:i+window_size]
        # добавим единичку всем парам слов в этом окне
        for j, k in combinations(window, 2):
            # чтобы граф был ненаправленный 
            m[j][k] += 1
            m[k][j] += 1
    
    # нормализуем строки, чтобы получилась вероятность перехода
    for i in range(m.shape[0]):
        m[i] /= np.sum(m[i])
    
    # случайно выберем первое слова, а затем будет выбирать на основе полученых распределений
    # сделаем так 5 раз и добавим каждое слово в счетчик
    # чтобы не забиться в одном круге, иногда будет перескакивать на случайное слово
    
    c = Counter()
    # начнем с абсолютного случайно выбранного элемента
    n = np.random.choice(len(vocab))
    for i in range(walks): # если долго считается, можно уменьшить число проходов
        
        # c вероятностью random_p 
        # перескакиваем на другой узел
        go_random = np.random.choice([0, 1], p=[1-random_p, random_p])
        if go_random:
            n = np.random.choice(len(vocab))
        
        n = take_step(n, m)
        # записываем узлы, в которых были
        c.update([n])
    
    # вернем топ-N наиболее часто встретившихся сл
    return [id2word[i] for i, count in c.most_common(top)]

def take_step(n, matrix):
    rang = len(matrix[n])
    # выбираем узел из заданного интервала, на основе распределения из матрицы совстречаемости
    next_n = np.random.choice(range(rang), p=matrix[n])
    return next_n
    


In [678]:
%%time
keywords_rw = data['content_norm'].apply(lambda x: get_kws(x, 10, 10))

CPU times: user 1min 50s, sys: 1.17 s, total: 1min 51s
Wall time: 1min 52s


In [679]:
evaluate(data['keywords'], keywords_rw)

Precision -  0.11
Recall -  0.21
F1 -  0.14
Jaccard -  0.08


In [42]:
keywords_rw.head(10)

0    [акция, яблоко, активист, год, деятельность, уважение, освобождение, обязательство, страна, политика]
1                      [газа, куб, метр, год, газпром, население, добыча, миллиард, производитель, страна]
2                               [год, роман, книга, поэзия, сюжет, работа, перевод, поэт, наследник, рука]
3                [ким, видео, зинаида, монтаж, процесс, подсудимая, сайт, свидетель, заседание, журналист]
4                            [га, площадь, столица, москва, парк, территория, парка, развитие, год, спорт]
5                          [гонка, позиция, место, этап, команда, строчка, чемпионат, балл, пилот, замена]
6                       [место, позиция, рф, влияние, глава, сергей, институт, россия, эксперт, политолог]
7           [закон, культура, сфера, внесение, концепция, услуга, сообщество, изменение, учреждение, срок]
8                               [газопровод, сирия, европа, газа, катар, турция, год, поток, россия, иран]
9                [участник, организац

Попбруем теперь важность считать с помощью какой-нибудь метрики из networkx.

In [684]:
import networkx as nx

In [725]:
def build_matrix(text, window_size=5):
    vocab = set(text)
    word2id = {w:i for i, w in enumerate(vocab)}
    id2word = {i:w for i, w in enumerate(vocab)}
    # преобразуем слова в индексы для удобства
    ids = [word2id[word] for word in text]

    # создадим матрицу совстречаемости
    m = np.zeros((len(vocab), len(vocab)))

    # пройдемся окном по всему тексту
    for i in range(0, len(ids), window_size):
        window = ids[i:i+window_size]
        # добавим единичку всем парам слов в этом окне
        for j, k in combinations(window, 2):
            # чтобы граф был ненаправленный 
            m[j][k] += 1
            m[k][j] += 1
    
    return m, id2word

def some_centrality_measure(text, window_size=5, topn=5):
    
    matrix, id2word = build_matrix(text, window_size)
    G = nx.from_numpy_array(matrix)
    # тут можно поставить любую метрику
    node2measure = dict(nx.degree(G))
    #print(node2measure)
    return [id2word[index] for index,measure in sorted(node2measure.items(), key=lambda x: -x[1])[:topn]]

In [726]:
%%time
keyword_nx = data['content_norm'].apply(lambda x: some_centrality_measure(x, 10, 10))

CPU times: user 25.2 s, sys: 379 ms, total: 25.6 s
Wall time: 26 s


In [702]:
evaluate(data['keywords'], keyword_nx)

Precision -  0.13
Recall -  0.25
F1 -  0.16
Jaccard -  0.09


Результаты не превосходят tfidf, но и не сильно уступают. Явно можно что-то доработать и превзойти baseline.

## Домашнее задание

В семинаре установлен такой бейзлан - F1 -  0.16 (не будем учитывать точность и полноту по отдельности и отбросим жаккара).

**Ваша задача - предложить 3 способа побить бейзлайн. **

Нет никаких ограничений кроме:

1) нельзя изменять метрику
2) решение должно быть воспроизводимым

В качестве ответа нужно предоставить jupyter тетрадку с экспериментами (обязательное условие!) и описать каждую из идей в форме - https://goo.gl/forms/Zb89yjXFr37EITMq1

Каждый реализованный и описанный способ оценивается в 3 балла. Дополнительный балл можно получить, если способы затрагивают разные аспекты решения (например, первая идея - улучшить нормализацию, вторая - улучшить способ представления текста в виде графа, третья - предложить способ удаления из топа идентичных ключевых слов (рф, россия)).

Можно использовать мой код как основу, а можно придумать что-то полностью другой.

Если у вас никак не получается побить бейзлайн вы можете предоставить реализацию и описание неудавшихся экспериментов (каждый оценивается в 0.5 баллов).

In [656]:
from tqdm import tqdm_notebook as tqdm
import nltk
import collections
import math
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string
from pymystem3 import Mystem
mystem = Mystem()
import itertools

# 1. Нормализация и нахождение tfidf

Для лемматизации использую mystem. Считаю, что он лучше лемматизирует наши данные.

In [291]:
def preprocessing(input_text, del_stopwords=True, del_digit=True):
    """
    :input: raw text
        1. lowercase, del punctuation, tokenize
        2. normal form
        3. del stopwords
        4. del digits
    :return: lemmas
    """
    russian_stopwords = set(stopwords.words('russian'))
    if del_digit:
        input_text = re.sub('[0-9]', '', input_text)
    words = [x.lower().strip(string.punctuation+'»«–…') for x in word_tokenize(input_text)]
    lemmas = [mystem.lemmatize(x)[0] for x in words if x]

    lemmas_arr = []
    for lemma in lemmas:
        if del_stopwords:
            if lemma in russian_stopwords:
                continue
        lemmas_arr.append(lemma)
    return lemmas_arr

Считаю tfidf по всему корпусу.

In [166]:
def compute_tf(text):
    tf_text = collections.Counter(text)
    for i in tf_text:
        tf_text[i] = tf_text[i]/float(len(text))
    return tf_text

In [167]:
def compute_idf(word, corpus):
    return math.log10(len(corpus)/sum([1.0 for i in corpus if word in i]))

In [168]:
def compute_tfidf(corpus):
    documents_list = []
    for text in tqdm(corpus):
        tf_idf_dictionary = {}
        computed_tf = compute_tf(text)
        for word in computed_tf:
            tf_idf_dictionary[word] = computed_tf[word] * compute_idf(word, corpus)
        documents_list.append(tf_idf_dictionary)
    return documents_list

#corpus = [['pasta', 'la', 'vista', 'baby', 'la', 'vista'], 
#['hasta', 'siempre', 'comandante', 'baby', 'la', 'siempre'], 
#['siempre', 'comandante', 'baby', 'la', 'siempre']]
#tfidf = compute_tfidf(corpus)

In [169]:
data['MY_lemmatized_text'] = data['content'].apply(preprocessing)

In [171]:
tfidf = compute_tfidf(data['MY_lemmatized_text'])

A Jupyter Widget

In [172]:
data['tfidf'] = tfidf

In [364]:
def order_by_tfidf(dict_lemma_tfidf):
    sorted_by_value = sorted(dict_lemma_tfidf.items(), key=lambda kv: kv[1], reverse=True)
    sorted_lemmas = [i[0] for i in sorted_by_value]
    return sorted_lemmas

In [365]:
data['order_by_tfidf'] = data['tfidf'].apply(order_by_tfidf)

In [424]:
def get_nouns(words, title=False):
    words = [morph.parse(word)[0] for word in words if word and word not in stops]
    words = [word.normal_form for word in words if word.tag.POS == 'NOUN']
    if title:
        return words
    else:
        return words[0:5]

In [419]:
data['MY_nouns_only'] = data['order_by_tfidf'].apply(get_nouns)

In [420]:
evaluate(data['keywords'], data['MY_nouns_only'])

Precision -  0.19
Recall -  0.18
F1 -  0.18
Jaccard -  0.11


##### Здесь я исправила только способ лемматизации и нахождение tfidf. Тут уже есть прирост, но поработаем ещё над title, так как часто слова из заголовка (или с начала текста, но это мы тут не учитываем) являются ключевыми.

# 2. Добавление слов из заголовка (доп. фичи)

Мы уже к найденным ключевым словам из текста добавляем все существительные из заголовка.

In [422]:
data['MY_lemmatized_title'] = data['title'].apply(preprocessing)

In [427]:
data['MY_lemmatized_title'] = data['MY_lemmatized_title'].apply(get_nouns, title=True)

In [428]:
data['title+text'] = data['MY_lemmatized_title'] + data['MY_nouns_only']

In [429]:
def to_set(l):
    l = list(set(l))
    return l

In [430]:
data['title+text'] = data['title+text'].apply(to_set)

In [431]:
evaluate(data['keywords'], data['title+text'])

Precision -  0.17
Recall -  0.24
F1 -  0.19
Jaccard -  0.12


##### Так ещё прирост!

# 3*. Важность имён

Хочется решить проблему "собянин vs дарья". Видим, что фамилии нам важны, а вот просто какие-то имена - не очень => уберем.

In [434]:
def get_nouns(words):
    words = [morph.parse(word)[0] for word in words if word and word not in stops]
    words = [word.normal_form for word in words if word.tag.POS == 'NOUN' and 'Name' not in word.tag]
    return words

In [435]:
get_nouns(['дарья', 'собянин', 'касперский', 'путин'])

['собянин', 'касперский', 'путин']

##### К сожалению, применив это для всех данных, мы не получили прирост (возможно, тут это не так важно).

# 4. Точный pos-tagging (pos -> normalization)

Я по-тупому определяла часть речи у леммы (не смотяр на контекст), но так не очень, так как, например "печь" = V или S). Нужно это исправить. Возьму текст и размечу его по частям речи и затем сделаю нормализацию.

In [488]:
def norm(text):
    info = mystem.analyze(text)
    lemmas = []
    russian_stopwords = set(stopwords.words('russian'))
    for w in info:
        try:
            analysis = w['analysis'][0]
            lex = analysis['lex']
            pos = re.search('(.+?),', analysis['gr']).group(1)
            if pos == 'S' and lex not in russian_stopwords:
                lemmas.append(lex)
        except:
            continue
    return lemmas

In [483]:
data['MY_lemmatized_text_2'] = data['content'].apply(norm)

In [491]:
tfidf2 = compute_tfidf(data['MY_lemmatized_text_2'])

A Jupyter Widget

In [492]:
data['tfidf2'] = tfidf2

In [493]:
data['order_by_tfidf_2'] = data['tfidf2'].apply(order_by_tfidf)

In [494]:
def get_first_5(words):
    return words[0:5]

In [495]:
data['MY_nouns_only_2'] = data['order_by_tfidf_2'].apply(get_first_5)

In [496]:
evaluate(data['keywords'], data['MY_nouns_only_2'])

Precision -  0.2
Recall -  0.2
F1 -  0.19
Jaccard -  0.12


##### Получился результат F1, как в пункте 2, но мы ещё не добавляли здесь заголовок.

In [497]:
data['MY_lemmatized_title_2'] = data['title'].apply(norm)

In [498]:
data['title+text_2'] = data['MY_lemmatized_title_2'] + data['MY_nouns_only_2']
data['title+text_2'] = data['title+text_2'].apply(to_set)

In [499]:
evaluate(data['keywords'], data['title+text_2'])

Precision -  0.18
Recall -  0.26
F1 -  0.21
Jaccard -  0.13


##### Получилось ещё лучше! Тут попробуем из итогового списка удалить имена.

In [540]:
def del_names(my_keys):
    without_names = []
    for every in my_keys:
        info = mystem.analyze(every)[0]['analysis'][0]['gr']
        #print(every, ':', info)
        if ',имя,' not in info:
            without_names.append(every)
    return without_names

In [541]:
del_names(['путин', 'ира', 'яблоко'])

['путин', 'яблоко']

In [542]:
data['title+text_2_without_names'] = data['title+text_2'].apply(del_names)

In [543]:
evaluate(data['keywords'], data['title+text_2_without_names'])

Precision -  0.19
Recall -  0.26
F1 -  0.21
Jaccard -  0.13


##### Тут только Precision немного поменялся

# 5*. Убираем частотные слова

Keywords не должны содержать самые частотные слова из корпуса - удалим их (то есть ключевые слова должны разделять тексты, а самые частотные не выделяют тексты).

In [596]:
def create_corpus(text, title, top=50):
    corpus = []
    for t in text:
        for w in t:
            if w not in stops:
                corpus.append(w)
    for t in title:
        for w in t:
            if w not in stops:
                corpus.append(w)
    counter_obj = Counter(corpus)
    return [e[0] for e in counter_obj.most_common(n=top)]

In [597]:
common = create_corpus(data['MY_lemmatized_text_2'], data['MY_lemmatized_title_2'], top=20)

In [598]:
def del_common(my_keys):
    without_common = []
    for every in my_keys:
        if every not in common:
            without_common.append(every)
    return without_common

In [599]:
data['title+text_2_without_common'] = data['title+text_2'].apply(del_common)

In [600]:
evaluate(data['keywords'], data['title+text_2_without_common'])

Precision -  0.18
Recall -  0.25
F1 -  0.2
Jaccard -  0.12


##### Уменьшило показатели - не сработало.

# 6. Графы: взвесим графы по tfidf

In [831]:
def build_matrix(text, window_size=5):
    vocab = set(text)
    word2id = {w:i for i, w in enumerate(vocab)}
    id2word = {i:w for i, w in enumerate(vocab)}
    # преобразуем слова в индексы для удобства
    ids = [word2id[word] for word in text]

    # создадим матрицу совстречаемости
    m = np.zeros((len(vocab), len(vocab)))

    # пройдемся окном по всему тексту
    for i in range(0, len(ids), window_size):
        window = ids[i:i+window_size]
        # добавим единичку всем парам слов в этом окне
        for j, k in combinations(window, 2):
            # чтобы граф был ненаправленный 
            m[j][k] += 1
            m[k][j] += 1
    
    return m, id2word

def some_centrality_measure(text, i, window_size=5, topn=5):
    
    matrix, id2word = build_matrix(text, window_size)
    #print(matrix, id2word)
    G = nx.from_numpy_array(matrix)
    #print(G)
    # тут можно поставить любую метрику
    node2measure = dict(nx.degree(G))
    #print(node2measure)
    for key in node2measure.keys():
        try:
            tf_idf = data['tfidf'][i][id2word[key]]
        except:
            tf_idf = 0
        #print(id2word[key], str(tf_idf))
        node2measure[key] = node2measure[key]/len(G)*tf_idf
    #print(node2measure)
    return [id2word[index] for index,measure in sorted(node2measure.items(), key=lambda x: -x[1])[:topn]]

In [832]:
i = 0
result = []
for text in data['content_norm']:
    keys = some_centrality_measure(text, i, window_size=5, topn=5)
    result.append(keys)
    i += 1

In [833]:
evaluate(data['keywords'], result)

Precision -  0.21
Recall -  0.2
F1 -  0.19
Jaccard -  0.12


##### Получился прирост!

# Результат:
## Увеличило F1:
1) **F1 = 0.18** Нормализация c помощью mystem и нахождение tfidf "вручную"

2) **F1 = 0.19** Добавление существительных из заголовка

3) **F1 = 0.19** Узлы графа, взвешанные по tfidf

4) **F1 = 0.21** Pos-tagging на основе контекста + добавление существительных из заголовка

## Никак не повлияло:

1) Удаление имён

2) Удаление частотных слов по корпусу
